# Training Embeddings con GNN
Aca crearemos embeddings paa un grafo de Internet, es decir represenaciones de los SA a partir de la topologia y atributos de los SA.

Para esto tomamos 3 enfoques:


*   [Caso 1] Reconstruction Approach autoencoder: A traves de la prediccion de aristas construimos los embeddings de los nodos.
*   [Caso 1.1] Reconstruction Approach autoencoder: en ves de ocupar las bgp routes recolectadas de los ribs ocupamos las relaciones de AS rank y le damso attr a las aristas (es solo para probar que lo estemos ahciendo bien y las cosas tienen sentido)
*   [Caso 2]: Reconstruction Approach attribute Masking:
*   Caso 3: Task Generation Pre-calculated descriptor

Para cada uno crearemos un ejemplo.


In [123]:
%load_ext autoreload
%autoreload 2
# from modules.gnn import GNN
import scipy.sparse as sp
import numpy as np
import os

from modules.graph import Graph, create_files
from modules.gnn import GNN
from modules.gnn_models import GCN, GraphSAGE

import torch
import torch.nn.functional as F

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [124]:
# Definimos las rutas de los archivos
base_path = os.getcwd() + "/data/"
relationships_file = base_path + "CAIDA_AS_Relationships/Serial_1/20220701.as-rel.txt.bz2"
features_file = base_path + "/node_features.csv"
rib_path = base_path + 'sanitized_rib.txt'
dataset_graph_path = base_path + 'dgl_graph/'

## Creacion Grafo 
Creamos un grafo nx y dgl, ademas de los archivos edges.csv y nodes.csv a partir de archivos ribs previamente creados o de archivo CAIDA AS Relationships.

Crear esos archivos una unica vez con create_graph() una ves ya creados los archivos edges.csv y nodes.csv puedo ocupar directamente la funcion 

Se le puede indicar el maximo de bgp paths que se quiere (hehco para cuando se leen ribs no de caida) 

### CASO 1: RIBs
* Creacion de grafo a partir de paths recolectados de las RIBs por BGPStream
* Por ahora le asignamos a todos los nodos embeddings iniciales de de dimension 32 parte con puros 1s todos


In [107]:
# Definimos las listas de features

LIST_FEATURES_NO_CATEG = ['ASN', 'AS_rank_numberAsns', 'AS_rank_numberPrefixes', 'AS_rank_numberAddresses', 'AS_rank_total', 'AS_rank_customer',
                 'AS_rank_peer', 'AS_rank_provider', 'peeringDB_ix_count', 'peeringDB_fac_count', 'AS_hegemony', 'cti_top', 'cti_origin']

LIST_FEATURES_CATEG = ['AS_rank_continent',
                        'peeringDB_info_ratio',
                        'peeringDB_info_scope',
                        'peeringDB_info_type',
                        'peeringDB_policy_general'
                        'ASDB_C1L1']

list_feat = LIST_FEATURES_NO_CATEG + LIST_FEATURES_CATEG

features_file = 'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
type = "DiGraph" # MultiDiGraph
relationships_file
max_paths = 100000
graph_case1 = create_files(type, 
            dataset_graph_path,
            file = rib_path, 
            features_file = features_file, 
            from_caida=False, 
            remove_degree=3,
            debug=True,
            max_paths = max_paths)
 
print('[NX Graph]',graph_case1.nx_graph)

[CaRPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[ARCHIVO EDGES.CSV CREADO]
[NX Graph]:  DiGraph with 36173 nodes and 75125 edges
[TOPOLOGIA CREADA]
[SAVE IN: /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/nodes.csv]
[FEATURES CREADOS]
[META CREADO]
[NX Graph]:  DiGraph with 14475 nodes and 53427 edges
[NX Graph]:  DiGraph with 13822 nodes and 52774 edges
[NX Graph]:  DiGraph with 13804 nodes and 52756 edges
[NX Graph] DiGraph with 13804 nodes and 52756 edges


### Caso 2
* Creacion del grafo a partir de CAIDA AS Relationships (AS Rank) 
* Se les da atributos a los edges correspondientes al tipo de relacion que comparten
* Es solo de prueba 

In [125]:
# Definimos las listas de features
LIST_FEATURES_NO_CATEG = ['ASN', 'AS_rank_numberAsns', 'AS_rank_numberPrefixes', 'AS_rank_numberAddresses', 'AS_rank_total', 'AS_rank_customer',
                 'AS_rank_peer', 'AS_rank_provider', 'peeringDB_ix_count', 'peeringDB_fac_count', 'AS_hegemony', 'cti_top', 'cti_origin']

LIST_FEATURES_CATEG = ['AS_rank_continent',
                        'peeringDB_info_ratio',
                        'peeringDB_info_scope',
                        'peeringDB_info_type',
                        'peeringDB_policy_general'
                        'ASDB_C1L1']

list_feat = LIST_FEATURES_NO_CATEG + LIST_FEATURES_CATEG

features_file =  'data/node_features.csv'  #'node_degrees' #'' # 'node_degrees' # las features que se le agregaran seran 
type = "DiGraph" # MultiDiGraph

max_paths = 100000
graph_case1 = create_files(type, 
             dataset_graph_path,
             relationships_file, 
             features_file, 
             from_caida=True, 
             remove_degree=3,
             debug=True,
max_paths = None)
 
print('[NX Graph]',graph_case1.nx_graph)

[CaRPETA CREADA]:  /home/valentina/Desktop/GIT/TrabajoTesis/data/dgl_graph/
[NX Graph]:  DiGraph with 74140 nodes and 379420 edges
[TOPOLOGIA CREADA]
[todos attr]
[bbbbbbbbbbbbbbbbbbbbbb]
[NX Graph]:  DiGraph with 74140 nodes and 379420 edges
[FEATURES CREADOS]
[META CREADO]
[NX Graph]:  DiGraph with 46714 nodes and 351994 edges
[NX Graph]:  DiGraph with 46237 nodes and 351517 edges
[NX Graph]:  DiGraph with 46229 nodes and 351509 edges
[NX Graph] DiGraph with 46229 nodes and 351509 edges


## Encode-Decoder 

In [126]:
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
from dgl.data import CoraGraphDataset
from dgl.dataloading import negative_sampler
import dgl.function as fn
from sklearn.metrics import roc_auc_score
from dgl.nn import SAGEConv, GraphConv, GATConv, GINConv, GatedGCNConv, GatedGraphConv

import numpy as np
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score



from dgl.nn import SAGEConv
from dgl.nn import SAGEConv, GraphConv, GATConv, GINConv, GatedGCNConv, GatedGraphConv
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools


In [127]:
def compute_auc(pos_score,neg_score):
    scores = torch.cat([pos_score, neg_score]).numpy()
    labels = torch.cat([torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]).numpy()
    return roc_auc_score(labels, scores)


def compute_loss(pos_score, neg_score):
    scores = torch.cat([pos_score, neg_score])
    labels = torch.cat(
        [torch.ones(pos_score.shape[0]), torch.zeros(neg_score.shape[0])]
    )
    return F.binary_cross_entropy_with_logits(scores, labels)


### Link Prediction
Encoder : GNN
Decoder : DotProduct

In [115]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

# FIXME: Cambiar, por mientras se estan agregando feat aleatorias o 1s a los nodos
# gnn.dgl_graph.ndata['feat'] = torch.ones(gnn.dgl_graph.num_nodes(), 128)            # features de tamano 128

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph(train_size=0.8)

Done saving data into cached files.
Graph(num_nodes=36173, num_edges=52756,
      ndata_schemes={'feat': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.float32)})
[ATTR SHAPE]:  torch.Size([36173, 2])
Generando 52756 aristas negativas...
Aristas negativas generadas: 52755


In [116]:

model = GraphSAGE(gnn.train_g.ndata["feat"].shape[1], 16,16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------- 4. training -------------------------------- #

all_logits = []
for e in range(100):
    # forward
    model.train()
    h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

    pos_score = model.decodeDotProduct(gnn.train_pos_g, h)
    neg_score = model.decodeDotProduct(gnn.train_neg_g, h)


    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = model.decodeDotProduct(gnn.test_pos_g, h)
    neg_score = model.decodeDotProduct(gnn.test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))


In epoch 0, loss: 2.2169077396392822
In epoch 5, loss: 0.8700197339057922
In epoch 10, loss: 0.5841309428215027
In epoch 15, loss: 0.5133039951324463
In epoch 20, loss: 0.48531290888786316
In epoch 25, loss: 0.468147873878479
In epoch 30, loss: 0.45938563346862793
In epoch 35, loss: 0.44982054829597473
In epoch 40, loss: 0.4390973448753357
In epoch 45, loss: 0.429028183221817
In epoch 50, loss: 0.42089155316352844
In epoch 55, loss: 0.41420429944992065
In epoch 60, loss: 0.409568727016449
In epoch 65, loss: 0.4064272940158844
In epoch 70, loss: 0.40386268496513367
In epoch 75, loss: 0.40169769525527954
In epoch 80, loss: 0.39999765157699585
In epoch 85, loss: 0.39872145652770996
In epoch 90, loss: 0.39759910106658936
In epoch 95, loss: 0.39672160148620605
AUC 0.956555513128636


### Link Prediction
Encoder : GNN
Decoder : MLP

In [128]:
gnn = GNN(debug=True)
gnn.load_dataset(dataset_graph_path, force_reload=True)

# FIXME: Cambiar, por mientras se estan agregando feat aleatorias o 1s a los nodos
# gnn.dgl_graph.ndata['feat'] = torch.ones(gnn.dgl_graph.num_nodes(), 128)            # features de tamano 128

print('[ATTR SHAPE]: ',gnn.dgl_graph.ndata['feat'].shape)

gnn.split_graph(train_size=0.8)


Done saving data into cached files.
Graph(num_nodes=74140, num_edges=351509,
      ndata_schemes={'feat': Scheme(shape=(72,), dtype=torch.float32)}
      edata_schemes={'Relationship': Scheme(shape=(), dtype=torch.int64)})
[ATTR SHAPE]:  torch.Size([74140, 72])
Generando 351509 aristas negativas...
Aristas negativas generadas: 351508


In [130]:
model = GraphSAGE(gnn.train_g.ndata["feat"].shape[1], 16,16)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# ----------- 4. training -------------------------------- #

all_logits = []
for e in range(100):
    # forward
    model.train()
    h = model.encode(gnn.train_g, gnn.train_g.ndata["feat"])

    pos_score = model.decodeMLP(gnn.train_pos_g, h)
    neg_score = model.decodeMLP(gnn.train_neg_g, h)


    loss = compute_loss(pos_score, neg_score)

    # backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 5 == 0:
        print("In epoch {}, loss: {}".format(e, loss))

# ----------- 5. check results ------------------------ #
from sklearn.metrics import roc_auc_score

with torch.no_grad():
    pos_score = model.decodeMLP(gnn.test_pos_g, h)
    neg_score = model.decodeMLP(gnn.test_neg_g, h)
    print("AUC", compute_auc(pos_score, neg_score))

In epoch 0, loss: 0.7252177000045776
In epoch 5, loss: 0.6580637097358704
In epoch 10, loss: 0.5534951090812683
In epoch 15, loss: 0.384759783744812
In epoch 20, loss: 0.244273841381073
In epoch 25, loss: 0.1576494425535202
In epoch 30, loss: 0.12169022113084793
In epoch 35, loss: 0.10285701602697372
In epoch 40, loss: 0.09458532184362411
In epoch 45, loss: 0.08908131718635559
In epoch 50, loss: 0.08555731177330017
In epoch 55, loss: 0.08315407484769821
In epoch 60, loss: 0.08149542659521103
In epoch 65, loss: 0.08005796372890472
In epoch 70, loss: 0.0789530873298645
In epoch 75, loss: 0.07803794741630554
In epoch 80, loss: 0.07713112235069275
In epoch 85, loss: 0.07627647370100021
In epoch 90, loss: 0.07546034455299377
In epoch 95, loss: 0.0746576339006424
AUC 0.9855221090672165


In [131]:
# Se incluyen los parametros tanto de MLP como de la GNN

for name, param in model.named_parameters():
    print(name, param.shape)


conv1.fc_neigh.weight torch.Size([16, 72])
conv1.fc_self.weight torch.Size([16, 72])
conv1.fc_self.bias torch.Size([16])
conv2.fc_neigh.weight torch.Size([16, 16])
conv2.fc_self.weight torch.Size([16, 16])
conv2.fc_self.bias torch.Size([16])
decoder.W1.weight torch.Size([16, 32])
decoder.W1.bias torch.Size([16])
decoder.W2.weight torch.Size([1, 16])
decoder.W2.bias torch.Size([1])


#### Guardar Modelo y Embeddings

In [132]:
# Guaradar el modelo
torch.save(model.state_dict(), 'model.pth')

# Guardar los embeddings
torch.save(h, "embeddings.pt")


In [133]:
import pandas as pd

# Guaradar el modelo
torch.save(model.state_dict(), 'model.pth')

# Guardar los embeddings de los nodos después del entrenamiento
with torch.no_grad():
    # Calcular los embeddings finales
    # final_embeddings = model.encode(train_g, train_g.ndata["feat"]).detach().cpu().numpy()
    final_embeddings = h.detach().cpu().numpy()

    # Crear un DataFrame para guardar los embeddings
    # Obtener ids de los nodos (ASN)
    # node_ids = train_g.nodes().numpy()
    node_ids = gnn.dgl_graph.nodes().numpy()

    emb_df = pd.DataFrame(final_embeddings, index=node_ids)
    emb_df.index.name = "node_id"

    # Guardar en un archivo CSV
    emb_df.to_csv("node_embeddings.csv")
    np.save("node_embeddings.npy", final_embeddings)
    print("Embeddings guardados en 'node_embeddings.csv'")

emb_df

Embeddings guardados en 'node_embeddings.csv'


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
node_id,,,,,,,,,,,,,,,,
0,2.960633,-0.530889,-0.063146,3.337903,-6.670915,2.733388,5.843852,3.043777,-2.202881,-10.431517,-1.514594,-6.829981,-8.230904,5.781105,2.572767,0.408480
1,3.474360,-0.500746,0.598364,3.052859,-6.689769,1.265362,5.260845,2.683862,-2.719274,-9.995238,-0.968039,-6.013503,-7.597056,5.277479,2.604416,-0.071746
2,3.691902,-0.606663,-1.135238,3.490635,-7.177549,3.193054,6.032834,3.134146,-2.324368,-10.792953,-2.193590,-8.526873,-9.285971,6.201936,2.646856,0.946869
3,2.065714,-0.306806,-1.747749,3.754204,-6.717747,3.283868,5.309824,3.119607,-0.934983,-10.517172,-2.546461,-7.387640,-7.450150,4.827490,1.935872,1.801739
4,2.342475,0.460370,-1.314342,2.776710,-5.806755,3.219721,7.075166,2.901008,-1.985440,-10.643597,-2.596992,-6.497208,-7.706456,4.182283,3.250054,-0.026504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74135,-0.644776,-0.340468,-0.916077,-0.612864,0.486238,0.940660,-1.089343,-0.368585,0.568008,0.174415,-0.495718,-1.449336,-0.532172,1.557938,-0.274276,1.011504
74136,4.297664,1.034229,-3.909623,3.547935,-7.930598,5.262478,6.749265,2.603194,0.681387,-11.181349,-4.115701,-10.065962,-7.557868,6.250222,2.218752,2.107581
74137,4.699925,1.784898,-6.673481,2.243041,-9.939831,5.294265,7.239003,2.592410,1.117161,-12.087595,-4.150962,-10.454863,-7.831730,6.572432,1.784056,2.385424


In [134]:
# Cargar embeddings
embeddings = np.load("node_embeddings.npy")
print("[SHAPE]",embeddings.shape)
print("[EMBEDDING]",embeddings[0])

[SHAPE] (74140, 16)
[EMBEDDING] [  2.9606333   -0.5308888   -0.06314588   3.3379025   -6.6709146
   2.733388     5.8438516    3.0437765   -2.2028809  -10.431517
  -1.5145942   -6.829981    -8.230904     5.781105     2.5727673
   0.4084804 ]


Cosas a tener consideraciom:
- GNN al crear un grafo, crea tdoos los nodos que se encuentran en el rango entregado. De esta forma pueden quedar nodos aislados, sin embargo ento causa que no se puedan ir 'actualizando' pero al agregar self_loop se actualizan con si mismos y supondremos que con ello algun patron para este tipo de nodos.
- La entrega de embeddings finales esta en orden el ASN de valor inferios hasta el ASN de valor maximo.
- Lo guardamso en un .csv dondde se asocia 'node_id' con su embeddings y donde node_id es el ASN del Sistema Autonomo. 

In [135]:
gnn.train_g.ndata['feat'] 

tensor([[0.1088, 0.3678, 0.4565,  ..., 0.0000, 0.0000, 0.0000],
        [0.1593, 0.4536, 0.5653,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3673, 0.6768,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0813, 0.2908,  ..., 0.0000, 0.0000, 0.0000],
        [0.1853, 0.3537, 0.5493,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0513, 0.2586,  ..., 0.0000, 0.0000, 0.0000]])

# TODO:
Primero:
* Crear grafo GNN a partir de AS rank y los atributos de ese otro dataset de antes.
* Con esa informacion entrenar los embedings y luego pasarle esos embedigs a a la clasificacion

Segundo:
* Probra crear grafo con archivos oix 
* Luego entrenar para clasificacion

Tercero:
* Copiar la Red Neuronal que ocupa BGP2Vec
* Probar con los embeddings anteriores probar la clasificacion. 
Si hay buenos resultados significa que nuestra MLP es muy basica/simple/poco compleja para atrapar las relaciones, identificar patrones. 
